In [22]:
import glob

read_files = glob.glob("*.rating")

with open("yelp.rating", "w") as outfile:
    for f in read_files:
        with open(f, "r") as infile:
            outfile.write(infile.read())

In [2]:
from collections import defaultdict
import numpy as np
import scipy, scipy.sparse, scipy.sparse.linalg
import matplotlib.pyplot as plt

In [4]:
graph = defaultdict(list)
users_set = set()
with open('yelp.links') as file:
    for line in file:
        node_1, node_2, _ = map(int, line.strip().split('\t'))
        graph[node_1].append(node_2)
        graph[node_2].append(node_1)
        users_set.update([node_1, node_2])
num_users = len(users_set)

NameError: name 'defaultdict' is not defined

In [66]:
items_set = set()
with open('yelp.rating') as file:
    for line in file:
        user, item, _ = map(int, line.strip().split('\t'))
        items_set.add(item)
num_items = len(items_set)
item_idx_encode = dict(zip(sorted(list(items_set)), 
                           range(num_users, num_users+num_items)))
with open('yelp.rating') as file:
    for line in file:
        user, item, _ = map(int, line.strip().split('\t'))
        item = item_idx_encode[item]
        graph[user].append(item)
        graph[item].append(user)

In [67]:
A = np.zeros((num_users+num_items, num_users+num_items))
for node in graph:
    for neighbor in graph[node]:
        A[node][neighbor] = 1
        A[neighbor][node] = 1

In [77]:
A_sparse = scipy.sparse.csc_matrix(A)

In [81]:
%store A_sparse

Stored 'A_sparse' (csc_matrix)


In [3]:
%store -r A_sparse

In [4]:
eig_val, eig_vec = scipy.sparse.linalg.eigs(A_sparse, k=1)

In [5]:
edge_score = eig_vec@eig_vec.T

In [6]:
edge_score = edge_score.real

In [7]:
edge_score = edge_score.astype('float32')

In [8]:
edge_score.min()

5.485221e-17

In [11]:
edge_score_min = 5.485221e-17

In [9]:
edge_score.max()

0.01299853

In [12]:
edge_score_max = 0.01299853

In [14]:
edge_score = (edge_score - edge_score_min)/(edge_score_max - edge_score_min)

In [15]:
import h5py

In [16]:
with h5py.File('yelp_edge_score.h5', 'w') as hf:
    hf.create_dataset("yelp_edge_score",  data=edge_score)